Q) The above data is data for prebid auctions. There are bidders who receive requests for placing ads, they respond to these requests with a bid and the highest bid wins the auction. Analyse the dataset to answer which bidder has the highest win rate (wins / total_bid).

## Import Libraries

In [1]:
import pandas as pd
import numpy as np

## Read CSV

In [2]:
bidding_data = pd.read_csv("analysis_id(in).csv", header = 0)
bidding_data.head()

,Unnamed: 0,date_hour,client,device_type,time_zone,os_name,browser_name,ad_unit,size,bidder,...,sum_time_to_respond,median,min_bid,max_bid,avg_bid,sum_2nd_highest_bid,sum_prebid_winning_bid,sum_winning_bid,sum_nobid,sum_timeout
0,0,2021-04-11T01:00:00,asc126828,Desktop,America/Chicago,Linux,Chrome-85.04183133,div-gpt-ad-BT-D-Footers,728x90,onemobile,...,6614.0,NaN,0.58520,0.58520,0.585200,0.5852,0.5852,NaN,NaN,NaN
1,1,2021-04-11T01:00:00,asc126828,Desktop,America/Chicago,Linux,Chrome-85.04183133,div-gpt-ad-BT-D-Footers,728x90,rhythmone,...,5171.0,NaN,0.33945,0.34875,0.344100,NaN,NaN,NaN,NaN,NaN
2,2,2021-04-11T01:00:00,asc126828,Desktop,America/Chicago,Linux,Chrome-85.04183133,div-gpt-ad-BT-D-InLine-1,300x250,undertone,...,1523.0,NaN,0.96000,0.96000,0.960000,NaN,NaN,NaN,NaN,NaN
3,3,2021-04-11T01:00:00,asc126828,Desktop,America/Chicago,Linux,Chrome-85.04183133,div-gpt-ad-BT-D-O-300x250-1,300x250,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2021-04-11T01:00:00,asc126828,Desktop,America/Chicago,Linux,Chrome-85.04183133,div-gpt-ad-BT-D-O-300x250-1,300x250,rhythmone,...,3570.0,0.34875,0.34410,0.34875,0.346425,NaN,NaN,NaN,NaN,NaN


## Analyse Data

In [3]:
bidding_data.columns.tolist() # All Columns

['Unnamed: 0',
 'date_hour',
 'client',
 'device_type',
 'time_zone',
 'os_name',
 'browser_name',
 'ad_unit',
 'size',
 'bidder',
 'bid_range',
 'media_type',
 'request_count',
 'response_count',
 'prebid_win_count',
 'win_count',
 'sum_bid',
 'sum_time_to_respond',
 'median',
 'min_bid',
 'max_bid',
 'avg_bid',
 'sum_2nd_highest_bid',
 'sum_prebid_winning_bid',
 'sum_winning_bid',
 'sum_nobid',
 'sum_timeout']

In [4]:
print('Shape of data: ', bidding_data.shape)

Shape of data:  (5000, 27)


In [5]:
bidding_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              5000 non-null   int64  
 1   date_hour               5000 non-null   object 
 2   client                  5000 non-null   object 
 3   device_type             5000 non-null   object 
 4   time_zone               5000 non-null   object 
 5   os_name                 5000 non-null   object 
 6   browser_name            5000 non-null   object 
 7   ad_unit                 5000 non-null   object 
 8   size                    5000 non-null   object 
 9   bidder                  5000 non-null   object 
 10  bid_range               5000 non-null   object 
 11  media_type              5000 non-null   object 
 12  request_count           5000 non-null   int64  
 13  response_count          5000 non-null   int64  
 14  prebid_win_count        1167 non-null   

Checking only relevant columns for this question: "client", "bidder","request_count","response_count", "prebid_win_count", "win_count"

In [6]:
bidding_data[["client", "bidder","request_count","response_count", "prebid_win_count", "win_count"]].isnull().sum()

client                 0
bidder                 0
request_count          0
response_count         0
prebid_win_count    3833
win_count           4814
dtype: int64

Although, Bidder is empty -> It is not showing above. Maybe, because of whitespaces.

In [7]:
bidding_data['bidder'].value_counts()

bidder
                1210
ix               464
onemobile        407
medianet         384
criteo           306
pubmatic         298
rhythmone        246
33across         240
triplelift       232
appnexus         197
openx            197
rubicon          178
emx_digital      169
sharethrough     139
pulsepoint       134
sovrn            102
undertone         49
teads             48
Name: count, dtype: int64

In [8]:
# Although bidder is empty, But it contains whitespaces
bidding_data['bidder'] = bidding_data['bidder'].apply(lambda x: x.strip())

In [9]:
bidding_data[["client", "bidder","request_count","response_count", "prebid_win_count", "win_count"]].isnull().sum()

client                 0
bidder                 0
request_count          0
response_count         0
prebid_win_count    3833
win_count           4814
dtype: int64

In [10]:
# Removing empty bidder rows
bidding_data = bidding_data[bidding_data['bidder'] != '']
print('Shape of the data after removing empty bidders: ',bidding_data.shape)

Shape of the data after removing empty bidders:  (3790, 27)


## Solution

I am taking sum of response_count as total_bid, as total_bids = Bidders actually replied with bids

In [11]:
#Top Bidders with most wins (sorted in descending order by win rate)
bidding_win_df = bidding_data.groupby( by = 'bidder').agg(win_count = ('win_count', 'sum'), total_bid = ('response_count','sum'))
bidding_win_df['win_rate'] = round(100 * bidding_win_df['win_count'] / bidding_win_df['total_bid'],2)
bidding_win_df = bidding_win_df.sort_values(by = 'win_rate', ascending=False).reset_index()
bidding_win_df

,bidder,win_count,total_bid,win_rate
0,undertone,16.0,104,15.38
1,emx_digital,20.0,393,5.09
2,sharethrough,10.0,332,3.01
3,rhythmone,18.0,655,2.75
4,pulsepoint,5.0,191,2.62
5,pubmatic,57.0,2335,2.44
6,teads,16.0,700,2.29
7,33across,19.0,886,2.14
8,onemobile,35.0,2726,1.28
9,appnexus,10.0,830,1.20


In [12]:
#Top Bidders with most prebid wins (sorted in descending order by win rate)
pre_bidding_win_df = bidding_data.groupby( by = 'bidder').agg(prebid_win_count = ('prebid_win_count', 'sum'), total_bid = ('response_count','sum'))
pre_bidding_win_df['win_rate'] = round(100 * pre_bidding_win_df['prebid_win_count'] / pre_bidding_win_df['total_bid'],2)
pre_bidding_win_df = pre_bidding_win_df.sort_values(by = 'win_rate', ascending=False).reset_index()

In [13]:
bidding_win_df.head(2) #Top 2 bidders with most bid win rate

,bidder,win_count,total_bid,win_rate
0,undertone,16.0,104,15.38
1,emx_digital,20.0,393,5.09


In [14]:
pre_bidding_win_df.head(2)  #Top 2 bidders with most pre-bid win rate

,bidder,prebid_win_count,total_bid,win_rate
0,undertone,52.0,104,50.00
1,teads,323.0,700,46.14


In [15]:
# For Theory Question
print('Odd Ratio - bid_amount:', round(np.exp(0.8),2))
print('Odd Ratio - time_to_bid:', round(np.exp(-0.4),2))
print('Odd Ratio - ad_unit_A: (reference category: ad_unit_B)', round(np.exp(0.3),2))

Odd Ratio - bid_amount: 2.23
Odd Ratio - time_to_bid: 0.67
Odd Ratio - ad_unit_A: (reference category: ad_unit_B) 1.35
